In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests

from pandas import CategoricalDtype
from bs4 import BeautifulSoup
from datetime import date
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from scipy import stats

import sys
import os

backend_path = os.path.abspath('../backend')
sys.path.append(backend_path)
from get_metrics import get_metrics

In [19]:
# чтение DataFrame df в файл data/df.csv
df = pd.read_csv('../data/df.csv')

# чтение DataFrame df_train в файл data/df_train.csv
df_train = pd.read_csv('../data/df_train.csv')

# чтение DataFrame df_test в файл data/df_test.csv
df_test = pd.read_csv('../data/df_test.csv')

# Baseline Prophet

In [20]:
# обучение модели baseline prophet
bl_prophet = Prophet()
bl_prophet.fit(df_train)

02:41:38 - cmdstanpy - INFO - Chain [1] start processing
02:41:39 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:
# смотрим совпадение фактического курса с предсказанным курсом baseline prophet
fig, ax = plt.subplots(figsize= (10, 5))

predict = bl_prophet.predict(df_test)
fig = bl_prophet.plot(predict, ax=ax)
ax.scatter(df_test.ds, df_test['y'], color = 'g')
ax.set_title('Прогноз ключевой ставки, Prophet')
ax.set_ylabel('Значение ставки')
ax.set_xlabel('Год')
plt.show()

In [ ]:
# смотрим тренд, годовые и сезонные признаки
fig = bl_prophet.plot_components(predict)
plt.show()

In [ ]:
# выводиим метрики mae и mape для оценкии baseline prophet 
metrics = get_metrics(y_test=df_test['y'], y_pred=predict['yhat'], name='Baseline Prophet')
metrics

1. Mean Absolute Error (MAE) - Среднее абсолютное отклонение позволяет оценить среднее абсолютное отклонение прогнозов от фактических значений. Чем ниже MAE, тем лучше модель.
2. Mean Absolute Percentage Error (MAPE) - Среднее абсолютное процентное отклонение показывает средний процент ошибки прогноза от фактических значений. Чем ниже MAPE, тем лучше.
3. Mean Squared Error (MSE) - Среднеквадратичное отклонение предоставляет информацию о среднеквадратичном отклонении прогнозов от фактических значений. Чем ниже MSE, тем лучше модель.
4. Root Mean Squared Error (RMSE) - Квадратный корень из MSE помогает интерпретировать стандартное отклонение прогнозов от фактических значений.

# Tuning Prophet

## Cross validation

In [ ]:
fix_percent = int(df.shape[0]*0.1)

In [ ]:
from prophet.diagnostics import cross_validation
df_cv = cross_validation(bl_prophet, initial='730 days', period='180 days', horizon = '365 days')

In [ ]:
fix_percent

In [ ]:
# выводиим метрики mae и mape для оценкии tuning prophet 
metrics = pd.concat([metrics, get_metrics(y_test=df_test['y'], y_pred=predict['yhat'], name='Tuning Prophet')])
metrics